## Imports | Parameters | Processors | Data
Do imports as shown below. 

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import torch.utils.data as data

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(0)

Parameters are set and can be changed freely. ``batchSize`` determines how much image is trained at once in the net. ``numEpochs`` determines how much training cycle is done i.e number of complete trainings of whole training data.

In [43]:
batchSize = 20
numEpochs = 3
learnRate = 0.005

Select GPU if available. M1 Mac uses ``mps``, regular nvidia uses ``cuda0``, otherwise ``cpu`` is selected

In [44]:
if torch.backends.mps.is_available:
    processor='mps'
elif torch.cuda.is_available():
    processor='cuda0'
else:
    processor='cpu'
device = torch.device(device=processor)
print(f"device is set as: {device}")

device is set as: mps


Get data CIFAR10 as pytorch can do it in the code. ``Normalize`` and randomly flip the training, testing sets then convert them to ``tensors``. There are also 10 classes as shown below. 

Normalization of data is done with respect to mean and std and the values are retrieved from: https://stackoverflow.com/questions/66678052/how-to-calculate-the-mean-and-the-std-of-cifar10-data

In [45]:
normalized = transforms.Normalize((0.49139968, 0.48215827 ,0.44653124), (0.24703233, 0.24348505, 0.26158768))
flip = transforms.RandomHorizontalFlip()
crop = transforms.RandomCrop(size=32)

trainValidTransform = transforms.Compose([transforms.ToTensor(), normalized, flip, crop])
testTransform = transforms.Compose([transforms.ToTensor(), normalized])

trainvalidSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=trainValidTransform)
trainSet , validSet = data.random_split(trainvalidSet, [35000, 15000])
testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=testTransform)

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True, num_workers=2)
validLoader = torch.utils.data.DataLoader(validSet, batch_size=batchSize, shuffle=True, num_workers=2)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print('Train data set:', len(trainSet))
print('Valid data set:', len(validSet))
print('Test data set:', len(testSet))

Files already downloaded and verified
Files already downloaded and verified
Train data set: 35000
Valid data set: 15000
Test data set: 10000


## The CNN Net | Training | Testing
The Net is defined as the class ``ConvNet`` and made as the ``model`` object. pytorch can show detailed summary for cuda0 or cpu processors but not for mps. Then for loss, ``CrossEntropyLoss`` is used since this net will clasify multiple clasess. As optimizer ``SGD`` i.e stochastic gradient descent used to optimize the parameters of the net.

The following guide has been adressed in creating a CNN architecture: https://www.youtube.com/watch?v=pDdP0TFzsoQ&t=251s

In [46]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=32, kernel_size=3, stride=1, padding=1) # 32,32,32 
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64, kernel_size=3, stride=1, padding=1) #64,32,32
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0) #64,16,16
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=94, kernel_size=3, stride=1, padding=1) #94,16,16
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0) #94,8,8

        self.fc1 = nn.Linear(in_features=94*8*8,out_features=128)
        self.fc2 = nn.Linear(in_features=128,out_features=64)
        self.fc3 = nn.Linear(in_features=64,out_features=10)
    
    def forward(self, x):
        x= F.relu(self.conv1(x))
        x= F.relu(self.conv2(x))
        x= self.pool1(x)
        x= F.relu(self.conv3(x))
        x= self.pool2(x)

        x= torch.flatten(x,start_dim=1)
        x= F.relu(self.fc1(x))
        x= F.relu(self.fc2(x))
        x= self.fc3(x)
        return x

model = ConvNet1().to(device)

if processor=='cpu' or processor=='cuda0':   
    from torchsummary import summary
    summary(model, (3, 32, 32))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learnRate, momentum=0.9)

The net is now put into the training with pulling batches from the trainLoader. Then the inputs need to be extracted according to the device selected to benefit the specific device support.

Then the net ``model`` is starting forward followed by back propagation of the error computed. The trained model is later saved to the directory of this file.

In [47]:
train_loss_hist = []
valid_loss_hist = []

train_acc_hist = []
valid_acc_hist = []

for epoch in range(numEpochs):
    train_loss = 0.0
    valid_loss = 0.0
    train_correct = 0.0
    train_total = 0.0
    valid_correct = 0.0
    valid_total = 0.0
    train_acc = 0.0
    valid_acc = 0.0
    model.train()
    for i, datatrain in enumerate(iterable=trainLoader, start=0):
        if processor == 'mps' or processor == 'cuda0': 
            inputs, labels = datatrain[0].to(device), datatrain[1].to(device)
        else:
            inputs, labels = datatrain
        optimizer.zero_grad()
        outputs = model(inputs)

        _,pred = torch.max(outputs,1)
        train_total += labels.size(0)
        train_correct += (pred == labels).sum()
        #print(f"Train acc status{train_total}, {train_correct}")

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(trainLoader)
    train_acc = 100* train_correct / train_total
    
    model.eval()
    for j, datavalid in enumerate(iterable=validLoader, start=0):
        if processor == 'mps' or processor == 'cuda0': 
            inputs, labels = datavalid[0].to(device), datavalid[1].to(device)
        else:
            inputs, labels = datavalid
        outputs = model(inputs)

        _,pred = torch.max(outputs,1)
        valid_total += labels.size(0)
        valid_correct += (pred == labels).sum()
        #print(f"Valid acc status{valid_total}, {valid_correct}")

        loss=criterion(outputs,labels)
        valid_loss+=loss.item()    
    valid_loss = valid_loss/len(validLoader)
    valid_acc = 100* valid_correct / valid_total


    print(f"For epoch: {epoch + 1} |---| trainloss={train_loss:.6f} validloss={valid_loss:.6f} | trainaccuracy={train_acc:.6f} valid={valid_acc:.6f}") 
    train_loss_hist.append(train_loss)
    valid_loss_hist.append(valid_loss)
    train_acc_hist.append(train_acc.cpu().numpy())
    valid_acc_hist.append(valid_acc.cpu().numpy())
print('Finished Training')
PATH = './cifar_net.v3.pth'
torch.save(model.state_dict(), PATH)

## Training loss vs Validation loss with vs number of epochs

In [ ]:
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.plot(train_loss_hist, label="training loss")
plt.plot(valid_loss_hist, label="validation loss")
plt.xlabel('epochs')
plt.ylabel('loss')
plt.title(f'loss of training and validation vs epochs({numEpochs})')
plt.legend(loc="upper right")

## Training accuracy vs Validation accuracy with vs number of epochs

In [ ]:
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.plot(train_acc_hist, label="training acc")
plt.plot(valid_acc_hist, label="validation acc")
plt.xlabel('epochs')
plt.ylabel('Accuracy %')
plt.title(f'loss of training and validation vs epochs({numEpochs})')
plt.legend(loc="upper left")

After The training, we now test the net ``model``. As we test the net, we disable the backpropagation and calculate the total percentage of succesfull outputs by comparing to their original label.

In the same loop, we also calculate the percentage of succesfull outputs of each class.

count=0
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data[0].to(device), data[1].to(device)
        count+=1
        #print(len(inputs), count)

        outputs = model(inputs)

In [ ]:
confusionMatrix = np.array([[0 for i in range(10)]]*10)
with torch.no_grad():
    correct = 0
    total = 0
    classCorrects = [0 for i in range(10)]
    classSamples = [0 for i in range(10)]
    for j, data in enumerate(iterable=testLoader, start=0):
        if processor == 'mps' or processor == 'cuda0': 
            inputs, labels = data[0].to(device), data[1].to(device)
        else:
            inputs, labels = data
        outputs = model(inputs)

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        for i in range(batchSize):
            label = labels[i]
            pred = predicted[i]
            confusionMatrix[labels[0].cpu().numpy(), predicted[0].cpu().numpy() ]+=1
            if (label == pred):
                classCorrects[label] += 1
            classSamples[label] += 1
    totalAcc=100 * correct / total
    print(f'Accuracy of the network on the {len(testLoader)*batchSize} test images: {totalAcc} %')
    for i in range(10):
        classAcc = 100 * classCorrects[i] / classSamples[i]
        print(f'Accuracy for class: {classes[i]} is {classAcc} %')
    

plt.rcParams["figure.figsize"] = [7.50, 3.50]      

## Example batch of 20 images with their actual and predicted values from training set

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(trainLoader)
images, labels = next(dataiter)

# show images
plt.grid(False)
imshow(torchvision.utils.make_grid(images))
# print labels
print('Actualval: '+' '.join(f'{classes[labels[j]]:5s} |' for j in range(batchSize)))

#print predictions
if processor == 'mps' or processor == 'cuda0': 
    outputs = model(images.to(device))
else:
    outputs = model(images)

_, predicted = torch.max(outputs, 1)
print('Predicted: '+' '.join(f'{classes[predicted[j]]:5s} |' for j in range(batchSize)))

## Plot Confusion Matrix with Actual vs Predicted counts

In [ ]:
plt.rcParams["figure.figsize"] = [10.50, 7.50]
plt.rcParams["figure.autolayout"] = True
fig, ax = plt.subplots()
for i in range(10):
   for j in range(10):
      c = confusionMatrix[i, j]
      ax.text(i, j, str(c), va='center', ha='center')
ax.matshow(confusionMatrix, cmap=plt.cm.Blues)
plt.xticks(range(10), classes, rotation=45)
plt.yticks(range(10), classes, rotation=45)


plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Training Confusion Matrix with epochs={numEpochs}')

ax.grid()
plt.rcParams["figure.autolayout"] = False